In [6]:
# Imports
import pandas as pd
import json
import re
import html

In [7]:
# Setup das variáveis
file_path = "C:\\Projetos Python\\Fiap\\F3-tech-challenge\\LF-Amazon-1.3M\\trn.json"
output_path = "C:\\Projetos Python\\Fiap\\F3-tech-challenge\\LF-Amazon-1.3M\\trn_tratado.json"
output_limited_path = "C:\\Projetos Python\\Fiap\\F3-tech-challenge\\LF-Amazon-1.3M\\trn_tratado_limitado.json"
output_limited_jsonl_path = "C:\\Projetos Python\\Fiap\\F3-tech-challenge\\LF-Amazon-1.3M\\trn_tratado_limitado.jsonl"
colunas_desejadas = ['title', 'content']
num_lines = 1000

# Passos:
 1. Escolhendo Dataset
 2. Preparação do Dataset - Limpeza do Dataset


In [8]:
def read_json_lines(file_path, num_lines=None, usecols=None, content_column='content'):
    """
    Lê um número específico de linhas de um arquivo JSON, ignora registros com conteúdo vazio e limpa caracteres especiais.

    Parâmetros:
    file_path (str): Caminho para o arquivo JSON.
    num_lines (int, opcional): Número de linhas a serem lidas. Se omitido, lê o arquivo inteiro.
    usecols (list, opcional): Lista de nomes de colunas a serem lidas. Se omitido, lê todas as colunas.
    content_column (str, opcional): Nome da coluna a ser verificada para conteúdo vazio e limpeza. Padrão é 'content'.

    Retorna:
    pd.DataFrame: DataFrame contendo as linhas lidas, filtradas e limpas.
    """
    if num_lines is None:
        df = pd.read_json(file_path, lines=True)
        if usecols:
            df = df[usecols]
        df = df[df[content_column].notna() & (df[content_column] != '')]
        df[content_column] = df[content_column].apply(clean_text)
        return df
    else:
        chunks = pd.read_json(file_path, lines=True, chunksize=num_lines)
        for chunk in chunks:
            if usecols:
                chunk = chunk[usecols]
            chunk = chunk[chunk[content_column].notna() & (chunk[content_column] != '')]
            chunk[content_column] = chunk[content_column].apply(clean_text)
            if not chunk.empty:
                return chunk
        return pd.DataFrame()  # Retorna um DataFrame vazio se nenhum chunk válido for encontrado



In [9]:
def save_json(df, output_path):
    """
    Salva o DataFrame em um arquivo JSON.

    Parâmetros:
    df (pd.DataFrame): DataFrame a ser salvo.
    output_path (str): Caminho para o arquivo de saída.
    """
    df.to_json(output_path, orient='records', lines=True)

In [10]:
def clean_text(text):
    """
    Remove entidades HTML e caracteres especiais de um texto.

    Parâmetros:
    text (str): Texto a ser limpo.

    Retorna:
    str: Texto limpo.
    """
    if isinstance(text, str):
        # Decodifica entidades HTML
        text = html.unescape(text)
        # Remove caracteres especiais, mantendo letras, números e espaços
        cleaned_text = re.sub(r'[^\w\s]', '', text)
        return cleaned_text
    return text

In [11]:
# Limpeza do dataframe original
df_tradado = read_json_lines(file_path, None, usecols=colunas_desejadas)
save_json(df_tradado, output_path)

In [12]:
# Limitar o número de linhas json para usar no modelo
df_tradado_limitado = read_json_lines(output_path, num_lines, None)
save_json(df_tradado_limitado, output_limited_path)

In [13]:
# Total de linhas do arquivo original
tamanho_original = len(pd.read_json(file_path, lines=True))
tamanho_tratado = len(df_tradado)
tamanho_tratado_limitado = len(df_tradado_limitado)

In [14]:
# Resultado da limpeza dos dados
# Limpeza feita:
# - Remoção de registros com conteúdo vazio
# - Leitura de um número específico de linhas
# - Limpar códigos HTML e caracteres especiais
# - Ignorar colunas que não serão usadas
print(f"A leitura de '{tamanho_original}' linhas do arquivo '{file_path}'")
print(f"Resultaram em '{len(df_tradado)}' linhas salvas no '{output_path}'.")
porc_linhas = (tamanho_tratado / tamanho_original) * 100
print(f"Isso representa '{porc_linhas:.2f}%' do arquivo original.")
print(f"Dataframe final:")
porc_linhas_limitado = (tamanho_tratado_limitado / tamanho_original) * 100
print(f"'{tamanho_tratado_limitado}' ('{porc_linhas_limitado:.2f}%') linhas salvas no '{output_limited_path}'.")


A leitura de '2248619' linhas do arquivo 'C:\Projetos Python\Fiap\F3-tech-challenge\LF-Amazon-1.3M\trn.json'
Resultaram em '1498718' linhas salvas no 'C:\Projetos Python\Fiap\F3-tech-challenge\LF-Amazon-1.3M\trn_tratado.json'.
Isso representa '66.65%' do arquivo original.
Dataframe final:
'1000' ('0.04%') linhas salvas no 'C:\Projetos Python\Fiap\F3-tech-challenge\LF-Amazon-1.3M\trn_tratado_limitado.json'.


In [15]:
import json

def convert_json_to_jsonl(input_file, output_file):
    """
    Converte um arquivo contendo objetos JSON separados por nova linha em um arquivo .jsonl.

    Args:
        input_file (str): Caminho para o arquivo de entrada contendo objetos JSON separados por nova linha.
        output_file (str): Caminho para o arquivo .jsonl de saída.
    """
    try:
        with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
            for line in f_in:
                line = line.strip()
                if line:  # Verifica se a linha não está vazia
                    try:
                        obj = json.loads(line)
                        json.dump(obj, f_out)
                        f_out.write('\n')
                    except json.JSONDecodeError:
                        print(f"Aviso: Linha inválida encontrada no arquivo {input_file}: {line}")
    except FileNotFoundError:
        print(f"Erro: Arquivo de entrada não encontrado: {input_file}")

# Exemplo de uso
input_file = 'seu_arquivo_de_entrada.json'  # Substitua pelo caminho do seu arquivo de entrada
output_file = 'seu_arquivo_de_saida.jsonl'

convert_json_to_jsonl(output_limited_path, output_limited_jsonl_path)

print(f"Arquivo .jsonl criado com sucesso: {output_limited_jsonl_path}")

Arquivo .jsonl criado com sucesso: C:\Projetos Python\Fiap\F3-tech-challenge\LF-Amazon-1.3M\trn_tratado_limitado.jsonl
